# QA 데이터셋 만들기 

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from typing import List
from pydantic import BaseModel, Field, HttpUrl

class Page(BaseModel):
    id: str = Field(..., description="ID of the Page")
    link: HttpUrl = Field(description="Url link of the page")
    name: str = Field(description="Name of the page")
    parent: str = Field(default="", description="ID of the parent page")
    child: List[str] = Field(default=[], description="List of ids of the child pages")
    description: str = Field(default="", description="Description of the page")
    description_clean: str = Field(default="", description="Content markdown")
    html_content: str = Field(default="", description="HTML code of the main content in the page")

    def __hash__(self):
        return hash((self.link, self.name))

    def __eq__(self, other):
        if not isinstance(other, Page):
            return False
        return (self.link, self.name) == (other.link, other.name)

## DB 가지고 오기 

In [3]:
import json

data_dir = '/Users/jwlee-pro/Documents/Workspace_2025/projects/llm-rag-chatbot/data/db/db-rngd_sdk.json'

with open(data_dir, "r") as f:
    data = json.load(f)
    final_pages = [Page.model_validate_json(page) for page in data["sdk"]]


In [4]:
final_pages[0]

Page(id='3e9b309f-d9d2-4ee7-be9e-9ffe421d4cfa', link=HttpUrl('https://furiosa-ai.github.io/docs-dev/2024.1/en/furiosa_llm/intro.html'), name='intro', parent='', child=[], description='\n\n\n\n\n* [.rst](../_sources/furiosa_llm/intro.rst "Download source file")\n* .pdf\n\n\n\n\n\n\n\n\n\n\nFuriosa LLM\n===========\n\n\n\n\n\nFuriosa LLM\n[#](#furiosa-llm "Link to this heading")\n====================================================\n\nFuriosa LLM provides a high-performance inference engine for LLM models and Multi-Modal LLM models,\nFuriosa LLM is designed to provide the state-of-the-art serving optimization.\nThe features of Furiosa LLM includes:\n\n* vLLM-compatible API\n* Efficient KV cache management with PagedAttention\n* Continuous batching of incoming requests in serving\n* Quantization: INT4, INT8, FP8, GPTQ, AWQ\n* Data Parallelism and Pipeline Parallelism across multiple NPUs\n* Tensor Parallelism (planned in release 2024.2) across multiple NPUs\n* OpenAI-compatible API server

In [5]:
# final_pages의 첫 번째 객체를 출력
print(final_pages[0])

# final_pages 전체 크기 확인
print(f"Total pages loaded: {len(final_pages)}")

id='3e9b309f-d9d2-4ee7-be9e-9ffe421d4cfa' link=HttpUrl('https://furiosa-ai.github.io/docs-dev/2024.1/en/furiosa_llm/intro.html') name='intro' parent='' child=[] description='\n\n\n\n\n* [.rst](../_sources/furiosa_llm/intro.rst "Download source file")\n* .pdf\n\n\n\n\n\n\n\n\n\n\nFuriosa LLM\n===========\n\n\n\n\n\nFuriosa LLM\n[#](#furiosa-llm "Link to this heading")\n====================================================\n\nFuriosa LLM provides a high-performance inference engine for LLM models and Multi-Modal LLM models,\nFuriosa LLM is designed to provide the state-of-the-art serving optimization.\nThe features of Furiosa LLM includes:\n\n* vLLM-compatible API\n* Efficient KV cache management with PagedAttention\n* Continuous batching of incoming requests in serving\n* Quantization: INT4, INT8, FP8, GPTQ, AWQ\n* Data Parallelism and Pipeline Parallelism across multiple NPUs\n* Tensor Parallelism (planned in release 2024.2) across multiple NPUs\n* OpenAI-compatible API server\n* Variou

## QA 데이터셋 만들기 

In [6]:
import os

# openAI key 
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

version #1
- html -> 마크다운 

In [7]:
from markdownify import markdownify as md
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.schema import HumanMessage
import pandas as pd

# LLM 설정
llm = ChatOpenAI(model="gpt-4o", temperature=0.2)

# 번역용 프롬프트 템플릿
prompt = PromptTemplate(template="""
You are an expert translator specializing in technical documentation. Your task is to translate the provided markdown content from English to Korean while retaining the original meaning and structure.

Instructions:
1. Translate all textual content to Korean.
2. Preserve programming code, CLI commands, and technical terms in English for readability.
3. Ensure the translated document is well-formatted in markdown style.
4. Avoid literal translations for idiomatic expressions; focus on clear, context-aware translations.

CONTENT:
{content}
""",
    input_variables=["content"],
)

# 번역 함수
def translate_documents(pages):
    """
    Translate the content of each page to Korean.

    Args:
        pages (List[Page]): List of Page objects.

    Returns:
        pd.DataFrame: DataFrame containing translated documents.
    """
    data = []

    for page in pages:
        try:
            # 프롬프트 생성
            prompt_message = prompt.format(content=page.description_clean)
            message = HumanMessage(content=prompt_message)

            # 번역 수행 (llm에 list 형태로 메시지 전달)
            translated_content = llm([message]).content

            # 데이터 추가
            data.append({
                "page_id": page.id,
                "link": str(page.link),
                "original_content": page.description_clean,
                "translated_content": translated_content,
            })
        except Exception as e:
            print(f"Error translating page {page.id}: {e}")

    # 번역 결과를 DataFrame으로 변환
    df = pd.DataFrame(data)
    return df



In [9]:
# 번역 실행하기 
translated_df = translate_documents(final_pages)

# 번역 결과를 CSV 및 JSON으로 저장
data_dir = '/Users/jwlee-pro/Documents/Workspace_2025/projects/llm-rag-chatbot/data/translate'
file_dir = f'{data_dir}/qa-rngd_sdk'

translated_df.to_csv(f"{file_dir}.csv", index=False, encoding="utf-8")
translated_df.to_json(f"{file_dir}.json", orient="records", force_ascii=False)

In [11]:
print(translated_df['translated_content'][1])

* [.rst](../../_sources/cloud_native_toolkit/kubernetes/device_plugin.rst "소스 파일 다운로드") * .pdf

Furiosa 디바이스 플러그인 설치

목차
--------

* [Furiosa 디바이스 플러그인](#furiosa-device-plugin)  
  + [설정](#configuration)  
  + [Helm을 사용한 Furiosa 디바이스 플러그인 배포](#deploying-furiosa-device-plugin-with-helm)

Furiosa 디바이스 플러그인 설치 [#](#installing-furiosa-device-plugin "이 제목으로 링크")

Furiosa 디바이스 플러그인 [#](#furiosa-device-plugin "이 제목으로 링크")
------------------------------------------------------------------------

Furiosa 디바이스 플러그인은 FuriosaAI NPU 장치를 위한 [Kubernetes 디바이스 플러그인](https://kubernetes.io/docs/concepts/extend-kubernetes/compute-storage-net/device-plugins/) 인터페이스를 구현하며, 그 기능은 다음과 같습니다:

* Furiosa NPU 장치를 발견하고 Kubernetes 클러스터에 등록합니다.
* 장치의 상태를 추적하고 Kubernetes 클러스터에 보고합니다.
* Kubernetes 클러스터 내에서 Furiosa NPU 장치 위에서 AI 작업을 실행합니다.

### 설정 [#](#configuration "이 제목으로 링크")

Furiosa NPU는 다양한 설정으로 Kubernetes 클러스터에 통합될 수 있습니다. 단일 NPU 카드는 하나의 리소스로 노출되거나 여러 리소스로 분할될 수 있습니다. 여러 리소스로 분할하면 더 세밀한 제어가 가능합니다.

다음 표는 사용 가능한 